# Pandas as a Database
Rather than installing postrgres and Django and configuring all that
You can just use a Pandas collection of DataFrame tables as your database
Lets see if we can find a "primary key" that we can use to connect a couple of these tables

In [93]:
import pandas as pd
pacs_scraped = pd.DataFrame.from_csv('public.raw_committees_scraped.csv')  # id
pacs = pd.DataFrame.from_csv('public.raw_committees.csv')  # no ID that I can find
candidates = pd.DataFrame.from_csv('public.raw_candidate_filings.csv')  # id_nmbr
print(pacs_scraped.info())
print(candidates.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1415 entries, The Good Government Council to Citizens Against Lawsuit Abuse PAC
Data columns (total 24 columns):
id                                     1415 non-null int64
acronym                                171 non-null object
pac_type                               415 non-null object
filing_effective_from                  1415 non-null object
filing_type                            1415 non-null object
address                                1415 non-null object
campaign_phone                         1238 non-null object
treasurer_name                         1415 non-null object
treasurer_mailing_address              1415 non-null object
treasurer_work_phone_home_phone_fax    1369 non-null object
treasurer_email_address                1412 non-null object
candidate_name                         766 non-null object
candidate_election_office              766 non-null object
candidate_party_affiliation            761 non-null object
candidat

# Primary Keys?
Find PK and foreign key fields to be able to join across tables

In [83]:
import re
from itertools import product
regex = re.compile(r'(\b|_|^)[Ii][Dd](\b|_|$)')
pac_id_cols = [col for col in pacs.columns if regex.search(col)]
print(pac_id_cols)
pac_scraped_id_cols = [col for col in pacs_scraped.columns if regex.search(col)]
print(pac_scraped_id_cols)
candidate_id_cols = [col for col in candidates.columns if regex.search(col)]
print(candidate_id_cols)
trans = pd.DataFrame.from_csv('public.raw_committee_transactions_ammended_transactions.csv')
trans_id_cols = [col for col in trans.columns if regex.search(col)]
print(trans_id_cols)
tables = [('pac', pacs, pac_id_cols), ('pac_scraped', pacs_scraped, pac_scraped_id_cols), ('candidate', candidates, candidate_id_cols), ('trans', trans, trans_id_cols)]
graph = []
for ((n1, df1, cols1), (n2, df2, cols2)) in product(tables, tables):
    if n1 == n2:
        continue
    for col1 in cols1:
        for col2 in cols2:
            s1 = set(df1[col1].unique())
            s2 = set(df2[col2].unique())
            similarity = float(len(s1.intersection(s2))) / float(len(s1.union(s2)))
            print('{}.{} -- {:.3} -- {}.{}'.format(n1, col1, similarity, n2, col2 ))
            graph += [(n1, col1, similarity, n2, col2)]
graph = pd.DataFrame(sorted(graph, key=lambda x:x[2]), columns=['table1', 'column1', 'similarity', 'table2', 'column2'])
print(graph)


[]
['id']
['id_nbr']
['original_id', 'contributor_payee_committee_id', 'filer_id']
pac_scraped.id -- 0.00114 -- candidate.id_nbr
pac_scraped.id -- 0.0 -- trans.original_id
pac_scraped.id -- 0.0283 -- trans.contributor_payee_committee_id
pac_scraped.id -- 0.169 -- trans.filer_id
candidate.id_nbr -- 0.00114 -- pac_scraped.id
candidate.id_nbr -- 0.0 -- trans.original_id
candidate.id_nbr -- 0.00239 -- trans.contributor_payee_committee_id
candidate.id_nbr -- 0.00908 -- trans.filer_id
trans.original_id -- 0.0 -- pac_scraped.id
trans.contributor_payee_committee_id -- 0.0283 -- pac_scraped.id
trans.filer_id -- 0.169 -- pac_scraped.id
trans.original_id -- 0.0 -- candidate.id_nbr
trans.contributor_payee_committee_id -- 0.00239 -- candidate.id_nbr
trans.filer_id -- 0.00908 -- candidate.id_nbr
         table1                         column1  similarity       table2  \
0   pac_scraped                              id    0.000000        trans   
1     candidate                          id_nbr    0.00

In [86]:
print(pacs_scraped.index.dtype)
print(pacs.index.dtype)

object
int64


In [87]:
trans = pd.DataFrame.from_csv('public.raw_committee_transactions_ammended_transactions.csv')
trans.describe()

,original_id,amount,aggregate_amount,contributor_payee_committee_id,filer_id,intrst_rate,check_nbr,addr_book_agent_name,zip,zip_plus_four,exp_date
count,2997.000000,2997.000000,2997.000000,160.000000,2997.000000,9,5.080000e+02,0,2355.000000,150.000000,0
mean,1885216.702369,1530.411655,4623.306210,7128.418750,9703.807808,0,5.985057e+06,NaN,90213.352017,5940.713333,NaN
std,211274.747411,10621.464988,30598.010565,6719.423466,6866.094987,0,1.331107e+08,NaN,21051.288729,2989.624421,NaN
min,71433.000000,-597.630000,0.000000,3.000000,10.000000,0,0.000000e+00,NaN,2117.000000,1.000000,NaN
25%,1864267.000000,35.000000,118.000000,629.000000,2480.000000,0,1.012750e+03,NaN,97038.000000,3498.000000,NaN
50%,1929490.000000,150.000000,250.000000,4749.000000,11508.000000,0,1.167500e+03,NaN,97219.000000,6628.000000,NaN
75%,1979015.000000,500.000000,977.650000,14931.000000,17001.000000,0,1.979250e+03,NaN,97401.000000,8652.000000,NaN
max,2049455.000000,452550.000000,543057.000000,17370.000000,17454.000000,0,3.000184e+09,NaN,99362.000000,9998.000000,NaN


# Original_ID?

So it looks like there are multiple revisions for many of the "unique" original_id. So to consolidate those revisions into unqiue records with unique IDs (take the most recent revision as the official record):

In [88]:
filtered_trans = []
for id in trans.original_id.unique():
    rows = sorted(trans[trans.original_id == id].iterrows(), key=lambda x:x[1].attest_date, reverse=True)
    filtered_trans += [rows[0][1]]
filtered_trans = pd.DataFrame(filtered_trans)
print(len(trans) / float(len(filtered_trans)))
print(filtered_trans.describe())

1.11
          original_id         amount  aggregate_amount  \
count     2700.000000    2700.000000       2700.000000   
mean   1884926.093333    1398.428496       4339.031659   
std     222078.767780   10643.772144      31247.914447   
min      71433.000000    -597.630000          0.000000   
25%    1864223.750000      35.000000        108.290000   
50%    1942541.000000     125.000000        238.805000   
75%    1979260.500000     500.000000        767.900000   
max    2049455.000000  452550.000000     543057.000000   

       contributor_payee_committee_id      filer_id  intrst_rate  \
count                      129.000000   2700.000000            9   
mean                      7872.085271   9612.684444            0   
std                       6991.062308   6897.686139            0   
min                          3.000000     10.000000            0   
25%                        470.000000   2189.000000            0   
50%                       4831.000000  11487.000000            0

In [89]:
df = filtered_trans
filer_sums = df.groupby('filer_id').amount.sum()
print(pacs_scraped.columns)
print(df.columns)
for (filer_id, amount) in sorted(filer_sums.iteritems(), key=lambda x:x[1], reverse=True):
    names = pacs_scraped[pacs_scraped.id == filer_id].index.values
    print('{}\t{}\t{}'.format(filer_id, names[0][:40] if len(names) else '', amount))


Index([u'id', u'acronym', u'pac_type', u'filing_effective_from',
       u'filing_type', u'address', u'campaign_phone', u'treasurer_name',
       u'treasurer_mailing_address', u'treasurer_work_phone_home_phone_fax',
       u'treasurer_email_address', u'candidate_name',
       u'candidate_election_office', u'candidate_party_affiliation',
       u'candidate_candidate_address', u'candidate_work_phone_home_phone_fax',
       u'candidate_mailing_address', u'candidate_email_address',
       u'candidate_occupation', u'candidate_employer', u'measure_election',
       u'measure_support', u'measure_details', u'committee_type'],
      dtype='object')
Index([u'original_id', u'tran_date', u'tran_status', u'filer',
       u'contributor_payee', u'sub_type', u'amount', u'aggregate_amount',
       u'contributor_payee_committee_id', u'filer_id', u'attest_by_name',
       u'attest_date', u'review_by_name', u'review_date', u'due_date',
       u'occptn_ltr_date', u'pymt_sched_txt', u'purp_desc', u'intrst_ra

# NLP
Let's build a graph of the similarity between PACs based on the wording of their committee names

In [90]:
import matplotlib
%matplotlib inline
np = pd.np
np.norm = np.linalg.norm
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split

In [101]:
df = pacs_scraped
names = df.index.values
corpus = [' '.join(str(f) for f in fields) for fields in zip(*[df[col] for col in df.columns if df[col].dtype == pd.np.dtype('O')])]
print(corpus[:3])
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 1), stop_words='english')
tfidf = vectorizer.fit_transform(corpus)
cov = tfidf * tfidf.T
cov[0:]


['GCC Miscellaneous 01/03/2014 to present Amendment 435 W First Ave P.O. Box 548 Albany, OR 97321 (541)926-1517 Janet Steele 435 1st Avenue W. Albany, OR 97321 (541)926-1517 (541)979-2395 (541)926-7064 jsteele@albanychamber.com nan nan nan nan nan nan nan nan nan nan nan nan PAC', 'nan nan 11/29/2012 to 11/29/2012 Discontinuation 28356 SW Wagner Street Wilsonville, OR 97070 (503)685-7346 Carol A. Russell 89358 Cranberry Lane Bandon, OR 97411 (800)893-1005 (541)347-4423 (800)967-7380 russell.cna@gmail.com Matt Wingard 2012 Primary Election State Representative, 26th District Republican 28356 SW Wagner St. Wilsonville, OR 97070 (503)351-2955 28356 SW Wagner St. Wilsonville, OR 97070 jobs_and_power@yahoo.com Public Relations Beambo, Inc., Wilsonville, OR nan nan nan CC', 'nan nan 04/30/2013 to present Amendment 3321 SE 20th Ave Portland, OR 97202 nan Kevin F. Neely PO Box 42307 Portland, OR 97242 (503)295-1851 kevin.neely@c-esystems.com John Kroger 2012 Primary Election Attorney General D

<1415x1415 sparse matrix of type '<type 'numpy.float64'>'
	with 2002225 stored elements in Compressed Sparse Row format>